In [1]:
conda activate pggb_fatica

(pggb_fatica) 


: 1

In [2]:
chromosomeName=$(odgi paths -L -i chr4_cluster.p95.s5000.n17.k47.G700-900-1100.at9852.pggbRecommended/chr4.fasta.gz.856e650.e34d4cd.a91cbe1.smooth.final.og|grep "9852")

(pggb_fatica) 


: 1

at9852_1_chr4:0-24203097
(pggb_fatica) 


: 1

## get length of each node


In [7]:
odgi view \
-i chr4_cluster.p95.s5000.n17.k47.G700-900-1100.at9852.pggbRecommended/chr4.fasta.gz.856e650.e34d4cd.a91cbe1.smooth.final.og \
-g \
-t 24 \
| grep '^S' \
| awk -v OFS='\t' '{print($2,length($3))}' \
> chr4_cluster.p95.s5000.n17.k47.G700-900-1100.at9852.pggbRecommended/chr4.fasta.gz.856e650.e34d4cd.a91cbe1.smooth.final.og.nodeLength.txt

echo -e "#node.id\tlength" > tmpHeader
cat tmpHeader chr4_cluster.p95.s5000.n17.k47.G700-900-1100.at9852.pggbRecommended/chr4.fasta.gz.856e650.e34d4cd.a91cbe1.smooth.final.og.nodeLength.txt > tmp 
mv tmp chr4_cluster.p95.s5000.n17.k47.G700-900-1100.at9852.pggbRecommended/chr4.fasta.gz.856e650.e34d4cd.a91cbe1.smooth.final.og.nodeLength.txt

(pggb_fatica) 
(pggb_fatica) 
(pggb_fatica) 
(pggb_fatica) 
(pggb_fatica) 


: 1

## get depth of each node

In [8]:
odgi \
    depth \
    -i chr4_cluster.p95.s5000.n17.k47.G700-900-1100.at9852.pggbRecommended/chr4.fasta.gz.856e650.e34d4cd.a91cbe1.smooth.final.og \
    -t 24 \
    -d \
    > chr4_cluster.p95.s5000.n17.k47.G700-900-1100.at9852.pggbRecommended/chr4.fasta.gz.856e650.e34d4cd.a91cbe1.smooth.final.og.nodeDepth.txt

(pggb_fatica) 


: 1

## merge depth and length

In [9]:
python mergeNodeLengthDepth.py \
-l chr4_cluster.p95.s5000.n17.k47.G700-900-1100.at9852.pggbRecommended/chr4.fasta.gz.856e650.e34d4cd.a91cbe1.smooth.final.og.nodeLength.txt \
-d chr4_cluster.p95.s5000.n17.k47.G700-900-1100.at9852.pggbRecommended/chr4.fasta.gz.856e650.e34d4cd.a91cbe1.smooth.final.og.nodeDepth.txt \
-o chr4_cluster.p95.s5000.n17.k47.G700-900-1100.at9852.pggbRecommended/chr4.fasta.gz.856e650.e34d4cd.a91cbe1.smooth.final.og.nodeDepthLength.txt

input length :chr4_cluster.p95.s5000.n17.k47.G700-900-1100.at9852.pggbRecommended/chr4.fasta.gz.856e650.e34d4cd.a91cbe1.smooth.final.og.nodeLength.txt
input depth : chr4_cluster.p95.s5000.n17.k47.G700-900-1100.at9852.pggbRecommended/chr4.fasta.gz.856e650.e34d4cd.a91cbe1.smooth.final.og.nodeDepth.txt
output file : chr4_cluster.p95.s5000.n17.k47.G700-900-1100.at9852.pggbRecommended/chr4.fasta.gz.856e650.e34d4cd.a91cbe1.smooth.final.og.nodeDepthLength.txt

 time for processing input: 3.283937454223633
(pggb_fatica) 


: 1

In [10]:
## here input should be a bed file of leftmost NLRs start and rightmost NLRs stop for the cluster (group of NLRs)
sed -i "s/at9852_1_chr4/${chromosomeName}/g" neighborhoods/chr4_compound.bed
while read chromosome start stop compoundName
do

    odgi \
    position \
    -i chr4_cluster.p95.s5000.n17.k47.G700-900-1100.at9852.pggbRecommended/chr4.fasta.gz.856e650.e34d4cd.a91cbe1.smooth.final.og \
    -t 24 \
    -b <(echo -e "${chromosome}\t${start}\t${stop}") \
    -v 

done < neighborhoods/chr4_compound.bed |cut -f1,4,5|cut -d ',' -f1,3|sed 's/+//g'|sed 's/,//g' > neighborhoods/chr4_compound.graph.bed

(pggb_fatica) 
(pggb_fatica) 
(pggb_fatica) 


: 1

cat: tmp1: No such file or directory
(pggb_fatica) 


: 1

In [11]:
rm tmp*
while read chromosome gstart gstop
do
    echo ${chromosome} >> tmp3
    
    # take fifth leftmost node with depth.uniq=18 and length greater than 100 as left anchor for the neighborhood
    awk '($2>100)&&($4==18)' chr4_cluster.p95.s5000.n17.k47.G700-900-1100.at9852.pggbRecommended/chr4.fasta.gz.856e650.e34d4cd.a91cbe1.smooth.final.og.nodeDepthLength.txt|awk -v "start=${gstart}" '($1<start)'|tail -n 5|head -n1|cut -f1 >> tmp1
    
    # take fifth rightmost node with depth.uniq=18 and length greater than 100 as right anchor for the neighborhood
    awk '($2>100)&&($4==18)' chr4_cluster.p95.s5000.n17.k47.G700-900-1100.at9852.pggbRecommended/chr4.fasta.gz.856e650.e34d4cd.a91cbe1.smooth.final.og.nodeDepthLength.txt|awk -v "stop=${gstop}" '($1>stop)'|head -n 5|tail -n1|cut -f1 >> tmp2

done < neighborhoods/chr4_compound.graph.bed

paste tmp3 tmp1 tmp2 > neighborhoods/chr4_neighborhood.graph.bed

(pggb_fatica) 
(pggb_fatica) 
(pggb_fatica) 
(pggb_fatica) 


: 1

In [ ]:
rm tmp*
while read chromosome nStart nStop
do
    echo ${chromosome} >> tmp3

    odgi position \
    -i chr4_cluster.p95.s5000.n17.k47.G700-900-1100.at9852.pggbRecommended/chr4.fasta.gz.856e650.e34d4cd.a91cbe1.smooth.final.og \
    -t 24 \
    -g ${nStart} \
    -r ${chromosome} |tail -n +2|cut -d ',' -f4,5|sed "s/,/\t/g"|cut -f1 >> tmp1

    odgi position \
    -i chr4_cluster.p95.s5000.n17.k47.G700-900-1100.at9852.pggbRecommended/chr4.fasta.gz.856e650.e34d4cd.a91cbe1.smooth.final.og \
    -t 24 \
    -g ${nStop} \
    -r ${chromosome} |tail -n +2|cut -d ',' -f4,5|sed "s/,/\t/g"|cut -f1 >> tmp2
done < neighborhoods/chr4_neighborhood.graph.bed

paste tmp3 tmp1 tmp2 > neighborhoods/chr4_neighborhood.coordinates.bed

In [14]:
inputOdgi=chr4_cluster.p95.s5000.n17.k47.G700-900-1100.at9852.pggbRecommended/chr4.fasta.gz.856e650.e34d4cd.a91cbe1.smooth.final.og
outdir=neighborhoods/chr4
count=0
while read a b c; 
do
    ((count+=1))   
    r_path=$(echo "${a}:${b}-${c}")
    r=${r_path}
    nh=$(echo "nh_${count}")
    outprefix=${nh}.region

    ########################## view the pangenome graph of the region ######################
    odgi viz \
    -i $inputOdgi \
    -o $outdir/${outprefix}.bm.png \
    -x 1000 \
    -bm \
    -t 20 \
    -P \
    -r ${r}
    ########################## Node Strandedness ############################################
    odgi viz \
    -i $inputOdgi \
    -o $outdir/${outprefix}.z.png \
    -x 1000 \
    -z \
    -t 20 \
    -P \
    -r ${r}
    
    ############################################ extract the region ############################################
    odgi extract \
    -i $inputOdgi \
    -r ${r} \
    --threads 24 -P \
    -d 100000 \
    -c 1  \
    -o $outdir/${outprefix}.og 
    
    ########################### get the coordinates in all the accessions ########################################
    odgi paths -L \
    -i $outdir/${outprefix}.og  \
    > $outdir/${outprefix}.allPathsCoordinates.txt
done < neighborhoods/chr4_neighborhood.coordinates.bed

(pggb_fatica) 
(pggb_fatica) 
(pggb_fatica) 
[odgi::viz] Path range to pangenomic range conversion.
[odgi::viz] Visualizing the graph in the pangenomic range [4.23956e+06, 4.43529e+06]
[odgi::viz] Binned mode
[odgi::viz] bin width: 195.735
[odgi::viz] image width: 1000
[odgi::viz] Path range to pangenomic range conversion.
[odgi::viz] Visualizing the graph in the pangenomic range [4.23956e+06, 4.43529e+06]
[odgi::viz] Binned mode
[odgi::viz] bin width: 195.735
[odgi::viz] image width: 1000
[odgi::extract] expansion and adding connecting edges
[odgi::extract] extracting path ranges 100.00% @ 1.02e+01/s elapsed: 00:00:00:00 remain: 00:00:00:00
[odgi::extract] merge subpaths closer than 100000 bps - iteration 1 (max 3) 100.00% @ 5.76e+00/s elapsed: 00:00:00:03 remain: 00:00:00:00
[odgi::extract] merge subpaths closer than 100000 bps - iteration 2 (max 3) 100.00% @ 7.48e+00/s elapsed: 00:00:00:02 remain: 00:00:00:00
[odgi::extract] adding connecting edges 100.00% @ 8.96e+04/s elapsed: 00:0

: 1

In [15]:
ls neighborhoods/chr4/nh_*.region.allPathsCoordinates.txt

neighborhoods/chr4/nh_1.region.allPathsCoordinates.txt
neighborhoods/chr4/nh_10.region.allPathsCoordinates.txt
neighborhoods/chr4/nh_11.region.allPathsCoordinates.txt
neighborhoods/chr4/nh_12.region.allPathsCoordinates.txt
neighborhoods/chr4/nh_13.region.allPathsCoordinates.txt
neighborhoods/chr4/nh_14.region.allPathsCoordinates.txt
neighborhoods/chr4/nh_15.region.allPathsCoordinates.txt
neighborhoods/chr4/nh_16.region.allPathsCoordinates.txt
neighborhoods/chr4/nh_2.region.allPathsCoordinates.txt
neighborhoods/chr4/nh_3.region.allPathsCoordinates.txt
neighborhoods/chr4/nh_4.region.allPathsCoordinates.txt
neighborhoods/chr4/nh_5.region.allPathsCoordinates.txt
neighborhoods/chr4/nh_6.region.allPathsCoordinates.txt
neighborhoods/chr4/nh_7.region.allPathsCoordinates.txt
neighborhoods/chr4/nh_8.region.allPathsCoordinates.txt
neighborhoods/chr4/nh_9.region.allPathsCoordinates.txt
(pggb_fatica) 


: 1

In [17]:
for i in {1..16}
do
sed -i "s/$/\tchr4_nh${i}/g" neighborhoods/chr4/nh_${i}.region.allPathsCoordinates.txt
done

for i in {1..9}
do 
sed -i "s/chr4_nh${i}/chr4_nh0${i}/g" neighborhoods/chr4/nh_${i}.region.allPathsCoordinates.txt
done


for i in {1..16}
do 
cut -d ":" -f1,3 neighborhoods/chr4/nh_${i}.region.allPathsCoordinates.txt |sed "s/:/\t/g"|sed "s/-/\t/g" > neighborhoods/chr4/nh_${i}.region.allPathsCoordinates.bed
done

(pggb_fatica) 


: 1